<a href="https://colab.research.google.com/github/malonsos/ClimateBert/blob/main/ClimateBert_Commitment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Commitments
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm
from google.colab import files

# Upload your Excel file to the Colab environment
uploaded = files.upload()

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel("classification_results_2010-item1.xlsx")

# Filter rows where label is "yes"
df_yes = df[df['Label'] == 'yes']

# Extract the text from the filtered DataFrame
texts = df_yes['Text'].tolist()

# Initialize model and tokenizer
model_name = "climatebert/distilroberta-base-climate-commitment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

# Initialize pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

# Process the texts through the pipeline


# Process the texts through the pipeline
results = []
for i, text in tqdm(enumerate(texts)):
    # Ensure text is a string
    if isinstance(text, str):
        out = pipe(text, padding=True, truncation=True)
        results.append({
            'File Name': df_yes.iloc[i]['File Name'],
            'Part Number': df_yes.iloc[i]['Part Number'],
            'Text': df_yes.iloc[i]['Text'],
            'Label': out[0]['label'],
            'Score': out[0]['score']
        })
    else:
        print(f"Ignoring non-string text at index {i}: {text}")





# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Add column counting words
results_df['WordClimate'] = results_df['Text'].apply(lambda x: len(str(x).split()))

# Save results to CSV file
results_df.to_csv("classification_commitment_2010-item1.csv", index=False)

# Download the CSV file
files.download("classification_commitment_2010-item1.csv")